In [108]:
(
exec 9<&1
ls -l /proc/$BASHPID/fd >>/tmp/sbash.log

exec 2>>/tmp/sbash.log
set -x

ls -l /proc/$BASHPID/fd >>/tmp/sbash.log

echopat='*echo $?*'

blockingread()
{
   while sleep 0.2; do
      IFS= read -r ln
      if [[ "$ln" == $echopat ]]; then
         echo 0 >&9  # because we changed fd1 and fd2
         echo -n "$PS1" >&9
         continue
      fi
      [ "$ln" = "" ] && continue
      break
   done
}

readcell() {
    blockingread
    echo "$ln"
    echo -n "$PS2">&9  # because we changed fd1 and fd2
    # read rest of the cell contents
    while IFS= read -t 0.2 -r lnr; do
      if [[ "$lnr" == $echopat ]]; then
         continue
      fi
      ## somehow get make this next line unnecessary!!!
      [[ "$lnr" == *endofcell* ]] && break
      echo "$lnr"
      echo -n "$PS2">&9  # because we changed fd1 and fd2
    done
    : return code $? period
}
   while IFS= read -t 0.5 -r ln; do
      echo "xxx,$ln"
   done
echo "Ready."

echo -n "$PS1"

while sleep 0.2; do
echo $$ >>/tmp/sbash.log
ls -l /proc/$BASHPID/fd >>/tmp/sbash.log
input="$(readcell)"
if [[ "$input" = *quit* ]]; then
  echo "bye"
  break
fi
echo a
echo "begin"
echo "$input"
echo "end"
echo
echo -n "$PS1"
#read ln3
#: "$ln3"
done
)

Ready.


In [109]:
echo \
  a \
  b
  endofcell

a
begin
echo \
  a \
  b
end



In [110]:
dateds
three
endofcell

a
begin
dateds
three
end



In [111]:
date
one
two
three
four
a
b
c
d
endofcell

a
begin
date
one
two
three
four
a
b
c
d
end



In [107]:
quit


bash: quit: command not found


In [1]:
export PS1 ; export PS2 ; while read -r ln ; do   echo "$ln" ; sleep 1 ; done | tee -a /tmp/ii | script /tmp/null | tee -a /tmp/oo